<a href="https://colab.research.google.com/github/JackBlake-zkq/robust-edge-inference/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from fa_ensemble import FiniteAggregationEnsemble
from torchvision.datasets import CIFAR10, MNIST
from torchvision import transforms
import torch
from torch import nn, optim
import random
import numpy
from tqdm import tqdm
import ssl
from models.resnet import ResNet18, ResNet18_1C

In [2]:
# trainset = CIFAR10(root='./data', train=True, download=True, transform=transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
#     transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ]))

# testset = CIFAR10(root='./data', train=False, download=True, transform=transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
# ]))
trainset = MNIST(root='./datasets/MNIST', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
testset = MNIST(root='./datasets/MNIST', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

In [3]:
def train_base_model(partition_number, train_subset):
    seed = partition_number
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    curr_lr = 0.1
    epochs = 5
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )

    trainloader = torch.utils.data.DataLoader(train_subset, batch_size=128, shuffle=True, num_workers=1)
    print("subset has ", len(train_subset), "data points")
    
    ssl._create_default_https_context = ssl._create_unverified_context
    net = ResNet18_1C() if isinstance(trainset, MNIST) else ResNet18()

    net = net.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(net.parameters(), lr=curr_lr, momentum=0.9, weight_decay=0.0005, nesterov= True)

    # Training
    net.train()
    for epoch in tqdm(range(epochs)):
        for (inputs, targets) in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        if (epoch in [60,120,160]):
            curr_lr = curr_lr * 0.2
            for param_group in optimizer.param_groups:
                param_group['lr'] = curr_lr


    net.eval()
    nomtestloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=1)
    correct = 0
    total = 0
    for (inputs, targets) in nomtestloader:
        inputs, targets = inputs.to(device), targets.to(device)
        with torch.no_grad():
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()
            total += targets.size(0)
        break
    acc = 100.*correct/total
    print(f'Estimated accuracy for base model {partition_number}: {str(acc)}%')

    return net


In [4]:
# ensemble = FiniteAggregationEnsemble(trainset, testset, train_base_model, 10, k=50, d=3, channels=1, state_dir="mnist_k50_d3")
# for i in range(150):
#     ensemble.train_base_model(i)
ensemble = FiniteAggregationEnsemble(trainset, testset, train_base_model, 10, k=10, d=3, channels=1, state_dir="mnist_k10_d3")
for i in range(30):
    ensemble.train_base_model(i)

Base model 0 already exists
Base model 1 already exists
Base model 2 already exists
Base model 3 already exists
Base model 4 already exists
Base model 5 already exists
Base model 6 already exists
Base model 7 already exists
Base model 8 already exists
Base model 9 already exists
Base model 10 already exists
Base model 11 already exists
Base model 12 already exists
Base model 13 already exists
Base model 14 already exists
Base model 15 already exists
Base model 16 already exists
Base model 17 already exists
Base model 18 already exists
Base model 19 already exists
Base model 20 already exists
Base model 21 already exists
Base model 22 already exists
Base model 23 already exists
Base model 24 already exists
Base model 25 already exists
Base model 26 already exists
Base model 27 already exists
Base model 28 already exists
Base model 29 already exists


In [5]:
ensemble.certify()

Certificates already computed


In [6]:
student = ResNet18_1C() if isinstance(trainset, MNIST) else ResNet18()
ensemble.distill(student)

  0%|          | 0/10 [00:18<?, ?it/s]


KeyboardInterrupt: 